In [33]:
import numpy as np
import torch
from torch import nn
from d2l import torch as d2l
n_train, n_test, num_inputs, batch_size = 20, 100, 200, 5
true_w, true_b = d2l.ones((num_inputs, 1)) * 0.01, 0.05   #0.01是权重,xi是一些随机的输入，噪声是均值为0，方差为0.01的正态分布
train_data = d2l.synthetic_data(true_w, true_b, n_train) #生成linear function of our inputs
train_iter = d2l.load_array(train_data, batch_size)
test_data = d2l.synthetic_data(true_w, true_b, n_test)
test_iter = d2l.load_array(test_data, batch_size, is_train=False)

In [34]:
# 初始化模型参数
def init_params():
    w=torch.normal(0,1,size=(num_inputs,1),requires_grad=True)
    b=torch.zeros(1,requires_grad=True)
    return [w,b]
# 定义L2范数惩罚
def l2_penalty(w):
    return torch.sum(w.pow(2))/2   # L2=1/2(w1**2+w2**2+...)
# 定义L1范数惩罚
def l1_penalty(w):
    return torch.sum(torch.abs(w))   # L1=\w1\+\w2\+...
# 定义训练代码
def train(lambd):
    w, b = init_params()
    net, loss = lambda X: d2l.linreg(X, w, b), d2l.squared_loss
    num_epochs, lr = 100, 0.003
    animator = d2l.Animator(xlabel='epochs', ylabel='loss', yscale='log',
                            xlim=[5, num_epochs], legend=['train', 'test'])
    for epoch in range(num_epochs):
        for X, y in train_iter:
            # 增加了L2范数惩罚项，
            # 广播机制使l2_penalty(w)成为一个长度为batch_size的向量
            l = loss(net(X), y) + lambd * l1_penalty(w)
            l.sum().backward()
            d2l.sgd([w, b], lr, batch_size)
        if (epoch + 1) % 5 == 0:
            animator.add(epoch + 1, (d2l.evaluate_loss(net, train_iter, loss),
                                     d2l.evaluate_loss(net, test_iter, loss)))
    print('w的L2范数是：', torch.norm(w).item())

In [35]:
train(lambd=0)  # 用lambd = 0禁用权重衰减后运行这个代码。 注意，这里训练误差有了减少，但测试误差没有减少， 这意味着出现了严重的过拟合。

w的L2范数是： 12.133252143859863


In [41]:
train(lambd=2)  # 下面，我们使用权重衰减来运行代码。 注意，在这里训练误差增大，但测试误差减小。 这正是我们期望从正则化中得到的效果。

w的L2范数是： 0.04793548583984375


In [31]:
# 简洁实现
def train_concise(wd):
    net = nn.Sequential(nn.Linear(num_inputs, 1))
    for param in net.parameters():
        param.data.normal_()
    loss = nn.MSELoss(reduction='none')
    num_epochs, lr = 100, 0.003
    # 偏置参数没有衰减
    trainer = torch.optim.SGD([
        {"params":net[0].weight,'weight_decay': wd},
        {"params":net[0].bias}], lr=lr)
    animator = d2l.Animator(xlabel='epochs', ylabel='loss', yscale='log',
                            xlim=[5, num_epochs], legend=['train', 'test'])
    for epoch in range(num_epochs):
        for X, y in train_iter:
            trainer.zero_grad()
            l = loss(net(X), y)
            l.mean().backward()
            trainer.step()
        if (epoch + 1) % 5 == 0:
            animator.add(epoch + 1,
                         (d2l.evaluate_loss(net, train_iter, loss),
                          d2l.evaluate_loss(net, test_iter, loss)))
    print('w的L2范数：', net[0].weight.norm().item())
    

In [32]:
train_concise(0)

w的L2范数： 12.067460060119629


In [24]:
train_concise(7)

w的L2范数： 0.08063127845525742


1.实验中权重衰减的值一般设置多少比较好？ 一般为e-2  、e-3
2.如果最优解的W就是比较大的数，那权重衰减是不是会有反作用？  不会，最优解是过拟合的W，，比较大，权重衰减会拉回正常的W
3.训练过程中,nan和inf是怎么产生的以及怎么解决？    nan是W除以0,inf是权重太大  解决：合理初始化权重  学习率不要太大
4.梯度消失是因为使用了sigmoid激活函数引起的吗？所以我们可以用Relu替换sigmoid解决梯度消失的问题？sigmoid容易造成梯度消失，Relu只会降低出现的概率
5.权重是在每个epoch结束以后更新的吧？   是在每个batchsize或iteration更新的